In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# データの読み込み
df = pd.read_csv('data/UNSW_NB15/UNSW_NB15_testing-set.csv')
df.head()

In [ ]:
# 各特徴量の分布を表示
df.hist(figsize=(24, 24))
plt.show()

In [ ]:
# 数値の列だけを取得
df_numeric = df.select_dtypes(include=[np.number])
df_numeric.drop('id', axis=1, inplace=True)

# 各特徴量の相関係数を表示
plt.figure(figsize=(24, 24))
sns.heatmap(df_numeric.corr(), annot=True, fmt='.2f')
plt.show()

In [ ]:
# レコード数が多いので、サンプリングする
df_numeric_sample = df_numeric.sample(n=10000, random_state=42)
sampled_indices = df_numeric_sample.index

# t-sneで二次元にプロット
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
tsne_result = tsne.fit_transform(df_numeric)


In [ ]:
tsne_result['attack_cat'] = df['attack_cat'].loc[sampled_indices]

plt.figure(figsize=(12, 12))
plt.title('t-SNE')
plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=attack_cat)
plt.colorbar(ticks=range(max(attack_labels)+1), label='attack')
plt.show()